In [1]:
import openpyxl
import pandas as pd

In [2]:
#対象のエクセルファイルをオープン
workbook_costdic = openpyxl.load_workbook('data_costdic.xlsx')
sheet_costdic = workbook_costdic["data"]

In [3]:
workbook_longterm = openpyxl.load_workbook('data_longterm.xlsx')
sheet_longterm = workbook_longterm["data"]

In [4]:
#計算対象となる設備の行数分計算
start_column = 19
number_column = 100
number_index = 300

In [5]:
for j in range(2,number_index):

    #更新年数の取得（設置年＋更新周期）
    renewal_year = sheet_longterm.cell(j,15).value + sheet_longterm.cell(j,17).value

    #更新年を記入
    sheet_longterm.cell(j,18).value = renewal_year

    #計算対象とする期間で計算
    for i in range(start_column,start_column + number_column):

        #更新費の算出
        cost = sheet_longterm.cell(j,13).value * sheet_longterm.cell(j,16).value

        #更新年が計算初年度を超過しているかを判定
        if sheet_longterm.cell(1,19).value > renewal_year:

            # 超過している場合は初年度に更新費を投入
            sheet_longterm.cell(j,19).value = cost
            
            # 残り更新回数
            renewal_times = number_column//sheet_longterm.cell(j,17).value

            # 更新年に金額を記入
            for k in range(1,renewal_times+1):
                sheet_longterm.cell(j,19 + sheet_longterm.cell(j,17).value*k).value = cost

            # 更新年以外に0を記入
            for k in range(20,start_column + number_column):
                if sheet_longterm.cell(j,k).value is None:
                    sheet_longterm.cell(j,k).value = 0
                    
            break    

        else:
            # 残り更新回数
            renewal_times = number_column//sheet_longterm.cell(j,17).value
            
            # 更新年数と計算対象年が一致すると記入
            for k in range(0,renewal_times):
                if sheet_longterm.cell(1,i).value == renewal_year + sheet_longterm.cell(j,17).value*k:
                    sheet_longterm.cell(j,i).value = cost
            
            # 更新年以外に0を記入
            for k in range(19,start_column + number_column):
                if sheet_longterm.cell(j,k).value is None:
                    sheet_longterm.cell(j,k).value = 0

In [6]:
print(sheet_longterm.cell(1,start_column + number_column).value)

None


In [7]:
rebuilding_year = 2060
rebuilding_cost = 9999

for i in range(2,number_index):
    
    # 更新費の算出
    cost = sheet_longterm.cell(i,13).value * sheet_longterm.cell(i,16).value

    # 残更新回数
    #renewal_times = sheet_longterm.cell(1,start_column + number_column).value - rebuilding_year
    renewal_times = 10
    
    # 建替年に建設費を記入
    for c in range(start_column,start_column + number_column):
        if sheet_longterm.cell(1,c).value == rebuilding_year:
            sheet_longterm.cell(i,c).value = rebuilding_cost
    
    # 建替以降の年数のコストを削除
    for c in range(start_column,start_column + number_column):
        if sheet_longterm.cell(1,c).value > rebuilding_year:
            sheet_longterm.cell(i,c).value = None

    # 建替年以降に更新費を記入    
    for k in range(1, renewal_times):
        for c in range(start_column,start_column + number_column):
            if sheet_longterm.cell(1,c).value == rebuilding_year + sheet_longterm.cell(i,17).value*k:
                sheet_longterm.cell(i,c).value = cost
            
    # 更新年以外に0を記入
    for c in range(start_column,start_column + number_column):
        if sheet_longterm.cell(i,c).value is None:
            sheet_longterm.cell(i,c).value = 0

In [8]:
workbook_longterm.save('toyo.xlsx')